In [1]:
pip install wandb

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import time
import torch

import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image

# import wandb

### GPU Setting ###
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda:0" if USE_CUDA else "cpu")
print(DEVICE)

cuda:0


In [4]:
pip install --upgrade pip

Note: you may need to restart the kernel to use updated packages.


In [1]:
"""#기본 베이스 코드.

### Custom Dataset ###
class CUB2011(Dataset):
    def __init__(self, transform, mode='train'):
        self.transform = transform
        self.mode = mode
        base_path = 'datasets'  # 기본 경로 설정

        if self.mode == 'train':
            folder_path = os.path.join(base_path, 'train')
        elif self.mode == 'valid':
            folder_path = os.path.join(base_path, 'valid')
        elif self.mode == 'test':
            folder_path = os.path.join(base_path, 'test')

        # 디렉토리 제외하고 파일만 리스트에 포함
        self.image_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        img_path = self.image_folder[idx]
        img = Image.open(os.path.join('datasets/', self.mode, img_path)).convert('RGB')
        img = self.transform(img)

        # 라벨을 이미지 파일명에서 추출 (예: 파일명이 'image_3.jpg'일 때 라벨은 3)
        label = img_path.split('_')[-1].split('.')[0]
        label = int(label)
        return img, label"""

"#기본 베이스 코드.\n\n### Custom Dataset ###\nclass CUB2011(Dataset):\n    def __init__(self, transform, mode='train'):\n        self.transform = transform\n        self.mode = mode\n        base_path = 'datasets'  # 기본 경로 설정\n\n        if self.mode == 'train':\n            folder_path = os.path.join(base_path, 'train')\n        elif self.mode == 'valid':\n            folder_path = os.path.join(base_path, 'valid')\n        elif self.mode == 'test':\n            folder_path = os.path.join(base_path, 'test')\n\n        # 디렉토리 제외하고 파일만 리스트에 포함\n        self.image_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]\n\n    def __len__(self):\n        return len(self.image_folder)\n\n    def __getitem__(self, idx):\n        img_path = self.image_folder[idx]\n        img = Image.open(os.path.join('datasets/', self.mode, img_path)).convert('RGB')\n        img = self.transform(img)\n\n        # 라벨을 이미지 파일명에서 추출 (예: 파일명이 'image_3.jpg'일 때 라벨은 3)\n        label 

In [4]:
import zipfile
import os

# 압축 파일과 압축 해제 경로
zip_file_path = 'CUB_200_2011_repackage_class50.zip'
extract_to = 'CUB_200_2011_class50'

# 폴더가 존재하지 않으면 생성
os.makedirs(extract_to, exist_ok=True)

# 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"'{zip_file_path}' 파일이 '{extract_to}' 폴더에 성공적으로 압축 해제되었습니다.")

'CUB_200_2011_repackage_class50.zip' 파일이 'CUB_200_2011_class50' 폴더에 성공적으로 압축 해제되었습니다.


# 학습 Schema 변형, 예시 데이터 증강 
- dataset 증강

In [3]:
### Custom Dataset ###
class CUB2011(Dataset):
    def __init__(self, transform, mode='train'):
        self.transform = transform
        self.mode = mode
        base_path = 'datasets'  # 기본 경로 설정

        if self.mode == 'train':
            folder_path = os.path.join(base_path, 'train')
        elif self.mode == 'valid':
            folder_path = os.path.join(base_path, 'valid')
        elif self.mode == 'test':
            folder_path = os.path.join(base_path, 'test')

        # 디렉토리 제외하고 파일만 리스트에 포함
        self.image_folder = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        img_path = self.image_folder[idx]
        img = Image.open(os.path.join('datasets/', self.mode, img_path)).convert('RGB')
        img = self.transform(img)

        # 라벨을 이미지 파일명에서 추출 (예: 파일명이 'image_3.jpg'일 때 라벨은 3)
        label = img_path.split('_')[-1].split('.')[0]
        label = int(label)
        return img, label

### Dataset 증강
- 크기 448 by 448 고정
- 무작위 회전
- 무작위 크롭
- 좌우 반전

In [4]:
from torch.utils.data import ConcatDataset, DataLoader
from torchvision import transforms

# 데이터 변환 정의
transforms_train = transforms.Compose([
    transforms.Resize((448, 448)),              # 크기 조정
    transforms.RandomRotation(15),              # 무작위 회전
    transforms.RandomResizedCrop(448, scale=(0.8, 1.0)),  # 무작위 크롭
    transforms.RandomHorizontalFlip(p=0.5),     # 좌우 반전
    transforms.ToTensor(),                      # 텐서 변환
])

transforms_valtest = transforms.Compose([
    transforms.Resize((448, 448)),              # 크기 조정
    transforms.ToTensor(),                      # 텐서 변환
])


In [6]:
from torch.utils.data import ConcatDataset, DataLoader

# 기본 Train 데이터셋
org_train_set = CUB2011(mode='train', transform=transforms_valtest)  # 기본 크기 조정만 적용

# 증강된 Train 데이터셋
transformed_train_set = CUB2011(mode='train', transform=transforms_train)

# 기본 데이터셋과 증강 데이터셋을 결합
train_set = ConcatDataset([org_train_set, transformed_train_set])

# Validation 및 Test 데이터셋
val_set = CUB2011(mode='valid', transform=transforms_valtest)
test_set = CUB2011(mode='test', transform=transforms_valtest)

# 데이터셋 크기 확인
print('Num of each dataset:', len(org_train_set), len(val_set), len(test_set))

print("Loaded combined dataloader")

Num of each dataset: 2360 296 298
Loaded combined dataloader


In [7]:
#Model List 선정
"""
1. EfficientNet_V2_M_Weights.IMAGENET1K_V1
2. ResNet50_Weights.IMAGENET1K_V2 : parameter수가 2배 차이나지만, 성능은 6퍼센트 높다
3. RegNet_Y_1_6GF_Weights_IMAGENET1K_V2 : ResNet18 parameter 수와 같은데 성능이 6퍼센트 높다.
4. ConvNext_Tiny_Weights.IMAGENET1K_V1 
"""
model_names = {
    "EfficientNet_V2_M": models.efficientnet_v2_m,
    "ResNet50": models.resnet50,
    "RegNet_Y_1_6GF": models.regnet_y_1_6gf,
    "ConvNeXt_Tiny": models.convnext_tiny
}

In [19]:
import wandb

### Train/Evaluation with wandb logging ###
def train(model, train_loader, optimizer, epoch):
    model.train()
    for i, (image, target) in enumerate(train_loader):
        image, target = image.to(DEVICE), target.to(DEVICE)
        output = model(image)
        optimizer.zero_grad()
        train_loss = F.cross_entropy(output, target).to(DEVICE)
        train_loss.backward()
        optimizer.step()

        # 매 10번째 배치마다 로그 남기기
        if i % 10 == 0:
            print(f'Train Epoch : {epoch} [{i}/{len(train_loader)}]\tLoss: {train_loss.item(): .6f}')
    return train_loss

def evaluate(model, val_loader):
    model.eval()
    eval_loss = 0
    correct = 0
    with torch.no_grad():
        for i, (image, target) in enumerate(val_loader):
            image, target = image.to(DEVICE), target.to(DEVICE)
            output = model(image)
            eval_loss += F.cross_entropy(output, target, reduction= 'sum').item()
            pred = output.argmax(1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    eval_loss /= len(val_loader.dataset)
    eval_accuracy = 100 * correct / len(val_loader.dataset)

    # validation 결과 wandb에 기록
    wandb.log({"val_loss": eval_loss, "val_accuracy": eval_accuracy})
    return eval_loss, eval_accuracy

In [20]:
# 하이퍼파라미터 스윕 설정
sweep_config = {
    'name' : 'No-Preprocess',
    'method': 'grid',
    'metric': {
        'name': 'val_accuracy',  # 최적화할 메트릭
        'goal': 'maximize'  # 성능을 최대화하려는 목표
    },
    'parameters': {
        'model_name': {
            'values': list(model_names.keys())
        },
        'learning_rate': {
            'values': [0.01, 0.001, 0.0001]  # 실험할 learning rate
        },
        'epochs': {
            'value': 50
        },
        'batch_size': {
            'values': [16, 32, 64]  # 실험할 배치 사이즈
        },
        'optimizer': {
            'values': ['sgd', 'rmsprop', 'adam']  # 사용할 optimizer
        }
    }
}

# Sweep 생성
sweep_id = wandb.sweep(sweep_config, project="resnet18")

Create sweep with ID: kq56ju9r
Sweep URL: https://wandb.ai/gachon-university-fdslab/resnet18/sweeps/kq56ju9r


In [21]:
# 학습 및 평가 함수
def train_and_evaluate(config=None):
    # wandb config 설정
    with wandb.init(config=config):
        config = wandb.config

        # 모델 로드
        print(f"")
        model_func = model_names[config.model_name]
        model = model_func(pretrained=True)
        
        # Transfer learning: 마지막 레이어 수정
        if hasattr(model, 'fc'):
            num_features = model.fc.in_features
            model.fc = nn.Linear(num_features, 50)
        else:
            num_features = model.classifier[1].in_features
            model.classifier[1] = nn.Linear(num_features, 50)

        # 모델 파라미터 설정
        params_to_update = []
        for param_name, param in model.named_parameters():
            if 'fc' in param_name or 'classifier.1' in param_name:
                param.requires_grad = True
                params_to_update.append(param)
            else:
                param.requires_grad = False

        model.to(DEVICE)
        
        if Preprocessing is True :
            train_loader = DataLoader(train_set, batch_size=config.batch_size, shuffle=True)
        else :
            train_loader = DataLoader(org_train_set, batch_size=config.batch_size, shuffle=True)

        val_loader = DataLoader(val_set, batch_size=config.batch_size, shuffle=False)
        test_loader = DataLoader(test_set, batch_size=config.batch_size, shuffle=False)

        # 옵티마이저 정의
        if config.optimizer == 'adam':
            optimizer = optim.Adam(params_to_update, lr=config.learning_rate)
        elif config.optimizer == 'sgd':
            optimizer = optim.SGD(params_to_update, lr=config.learning_rate)
        elif config.optimizer == 'rmsprop':
            optimizer = optim.RMSprop(params_to_update, lr=config.learning_rate)
        
        best_val_accuracy = 0
        for epoch in range(config.epochs):
            train_loss = train(model, train_loader, optimizer, epoch)
            val_loss, val_accuracy = evaluate(model, val_loader)

            # 최고 성능 모델 저장
            if val_accuracy > best_val_accuracy:
                best_val_accuracy = val_accuracy
                torch.save(model.state_dict(), f"./best_model_{config.model_name}.pth")
                print(f'Best model saved for {config.model_name} with accuracy: {best_val_accuracy:.4f}')

            # Log epoch results
            wandb.log({"best_val_accuracy": best_val_accuracy, "val_loss": val_loss, "epoch": epoch})

        # 최종 테스트 평가
        test_loss, test_accuracy = evaluate(model, test_loader)
        print(f'[FINAL] {config.model_name} Test Loss: {test_loss:.4f}, Accuracy: {test_accuracy:.4f}')
        wandb.log({"test_loss": test_loss, "test_accuracy": test_accuracy})

In [ ]:
# Sweep agent 
Preprocessing = False

wandb.agent(sweep_id, train_and_evaluate)

wandb: Agent Starting Run: rb3ueusp with config:
wandb: 	batch_size: 16
wandb: 	epochs: 50
wandb: 	learning_rate: 0.01
wandb: 	model_name: EfficientNet_V2_M
wandb: 	optimizer: sgd



Train Epoch : 0 [0/148]	Loss:  3.924208
Train Epoch : 0 [10/148]	Loss:  3.883040
Train Epoch : 0 [20/148]	Loss:  3.895839
Train Epoch : 0 [30/148]	Loss:  3.813031
Train Epoch : 0 [40/148]	Loss:  3.814166
Train Epoch : 0 [50/148]	Loss:  3.736363
Train Epoch : 0 [60/148]	Loss:  3.631399
Train Epoch : 0 [70/148]	Loss:  3.578503
Train Epoch : 0 [80/148]	Loss:  3.431796
Train Epoch : 0 [90/148]	Loss:  3.619381
Train Epoch : 0 [100/148]	Loss:  3.449720
Train Epoch : 0 [110/148]	Loss:  3.417991
Train Epoch : 0 [120/148]	Loss:  3.449693
Train Epoch : 0 [130/148]	Loss:  3.300104
Train Epoch : 0 [140/148]	Loss:  3.088228
Best model saved for EfficientNet_V2_M with accuracy: 58.7838
Train Epoch : 1 [0/148]	Loss:  3.054585
Train Epoch : 1 [10/148]	Loss:  3.096778
Train Epoch : 1 [20/148]	Loss:  2.896196
Train Epoch : 1 [30/148]	Loss:  2.875313
Train Epoch : 1 [40/148]	Loss:  2.921327
Train Epoch : 1 [50/148]	Loss:  2.510246
Train Epoch : 1 [60/148]	Loss:  2.522558
Train Epoch : 1 [70/148]	Loss:  